In [5]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats


from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer


from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

%matplotlib inline

In [6]:
class Categorical_Imputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
        #self.strategy = strategy


    def fit_transform(self,X,y=None):
        col_all = X.columns
        for col in col_all:
            #print(col,' ',X[col].value_counts().index[0])
            X[col].fillna(X[col].value_counts().index[0],inplace=True)
            #X.info()
        return X
 

In [7]:
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

y_train = train_df['SalePrice']
train_df = train_df.drop(['Id','SalePrice'],axis=1)
test_df = test_df.drop('Id',axis=1)



test_length = len(test_df)
train_length = len(train_df)

In [8]:
all_sample = pd.concat([train_df,test_df])
all_sample['MSSubClass'] = pd.Categorical(all_sample.MSSubClass)
num_col = list(all_sample._get_numeric_data().columns)
cat_col = list(set(all_sample.columns) - set(num_col))


In [9]:
# select numeric
select_numeric = FunctionTransformer(lambda x : x[num_col],validate=False)

In [10]:
select_cate = FunctionTransformer(lambda x : x[cat_col],validate=False)

In [11]:
cate_imputer = Categorical_Imputer()
cate_imp = FunctionTransformer(cate_imputer.fit_transform,validate=False)

In [12]:
get_dummies = FunctionTransformer(lambda x: pd.get_dummies(x, prefix_sep='_', drop_first=True), 
                                  validate=False)

Create a pipeline to handle the numeric data
steps
- Select
- imputer (simple mode Imputer is used now. A class can be used in the future)
- feature enginering (optional in the future)

In [13]:
#simple mode for the imputer. A class can be written in the future
num_imputer = SimpleImputer(strategy="most_frequent")
num_standard = StandardScaler()


num_pipeline = Pipeline(steps=
                        [('select_num',select_numeric),
                         ('imp_num',num_imputer),
                        ('standadize',num_standard)])
num_pipeline.fit_transform(all_sample)

array([[-0.12767816, -0.21787869,  0.6461834 , ..., -0.08959196,
        -1.55218353,  0.15764553],
       [ 0.56712524, -0.07204408, -0.06318454, ..., -0.08959196,
        -0.44692486, -0.60296158],
       [ 0.01128252,  0.13719688,  0.6461834 , ..., -0.08959196,
         1.02675338,  0.15764553],
       ...,
       [ 4.27274338,  1.24680804, -0.77255247, ..., -0.08959196,
         1.02675338, -1.36356868],
       [-0.26663884,  0.0346054 , -0.77255247, ...,  1.14431218,
         0.28991426, -1.36356868],
       [ 0.28920388, -0.06862014,  0.6461834 , ..., -0.08959196,
         1.7635925 , -1.36356868]])

create a pipeline to handle categorical data
steps
    - select
    - imputer
    - get_dummy

In [16]:
cate_pipeline = Pipeline(steps=[
                                ('select_cate',select_cate),
                                ('imp_cate',cate_imp),
                                ('get_dummy',get_dummies)
])
cate_pipeline.fit_transform(all_sample)

,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,...,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,PoolQC_Fa,PoolQC_Gd,LotShape_IR2,LotShape_IR3,LotShape_Reg
0,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,1
1,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,1
2,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
3,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
4,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
5,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
6,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,1
7,0,0,0,1,1,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
8,1,0,0,0,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0,0,1
9,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,1


In [19]:
#add cate and numeric
union = ColumnTransformer([
    ('numerical',num_pipeline,num_col),
    ('categorical',cate_pipeline,cat_col)
])
union_result = union.fit_transform(all_sample)

In [20]:
union_result.shape

(2919, 259)

In [21]:
X_train = union_result[:train_length]
X_test  = union_result[train_length:]

In [23]:

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

#xgb_Reg = xgb.XGBRegressor()



#Dtree_reg = DecisionTreeRegressor(random_state=42)

#ridgeReg = Ridge(alpha=10,normalize=True)
#lasso_Reg = Lasso(alpha=2,normalize=True)
#EN_Reg = ElasticNet(alpha=5,l1_ratio=0.5,normalize=False)


models = {
    'RidgeRegression':Ridge(),
    'LassoRegression':Lasso(),
    'ElasticNetRegression':ElasticNet(),
    'XBoostRegression':XGBRegressor()
}


params = {
    'RidgeRegression':{
        "alpha":[1,5,10,20],
        "normalize":[True,False]
    },
    'LassoRegression':{
        "alpha":[1,5,10,20],
        "normalize":[True,False]
    },
    'ElasticNetRegression':{
        "alpha":[1,5,10,20],
        "normalize":[True,False],
        "l1_ratio":[0.5]
    },
    'XBoostRegression':{
        "learning_rate":[0.05,0.1],
        "n_estimators":[10,50,100]
    }
    
    
}

In [26]:
from sklearn.model_selection import GridSearchCV

def model_fit(train_features, train_actuals):
        """
        fits the list of models to the training data, thereby obtaining in each 
        case an evaluation score after GridSearchCV cross-validation
        """
        for name in models.keys():
            est = models[name]
            est_params = params[name]
            gscv = GridSearchCV(estimator=est, param_grid=est_params, cv=5,
                               scoring='neg_mean_squared_error', return_train_score=True)
            gscv.fit(train_actuals, train_features)
            cvres = gscv.cv_results_
            print("best parameters are: {}".format(gscv.best_estimator_))
            for mean_score,par in zip(cvres["mean_test_score"],cvres["params"]):
                print(np.sqrt(-mean_score), par)
  
            
            
            
model_fit(y_train,X_train)


best parameters are: Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)
33712.545849222675 {'alpha': 1, 'normalize': True}
33544.914724393246 {'alpha': 1, 'normalize': False}
41581.47677830166 {'alpha': 5, 'normalize': True}
32414.677848614796 {'alpha': 5, 'normalize': False}
48402.70798148149 {'alpha': 10, 'normalize': True}
32228.473454633797 {'alpha': 10, 'normalize': False}
56604.33957280057 {'alpha': 20, 'normalize': True}
32281.18100138517 {'alpha': 20, 'normalize': False}


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2286704811.4073486, tolerance: 759214014.2531197
  positive)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2779334910.7435913, tolerance: 728773513.025937
  positive)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2679719090.6730347, tolerance: 699031751.5199751
  positive)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2315244204

best parameters are: Lasso(alpha=20, copy_X=True, fit_intercept=True, max_iter=1000, normalize=True,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)
43508.74873809707 {'alpha': 1, 'normalize': True}
44764.96588117624 {'alpha': 1, 'normalize': False}
38696.06128611283 {'alpha': 5, 'normalize': True}
42556.65825128134 {'alpha': 5, 'normalize': False}
35620.42940772728 {'alpha': 10, 'normalize': True}
40265.81610382474 {'alpha': 10, 'normalize': False}
33183.26071014407 {'alpha': 20, 'normalize': True}
36836.23259037077 {'alpha': 20, 'normalize': False}
best parameters are: ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
78036.91093277342 {'alpha': 1, 'l1_ratio': 0.5, 'normalize': True}
35329.46788393932 {'alpha': 1, 'l1_ratio': 0.5, 'normalize': Fal

//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:43:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[12:43:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:43:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
//anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[12:43:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best parameters are: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
123994.15915505907 {'learning_rate': 0.05, 'n_estimators': 10}
36868.822537178996 {'learning_rate': 0.05, 'n_estimators': 50}
29293.614173908118 {'learning_rate': 0.05, 'n_estimators': 100}
79207.16810334758 {'learning_rate': 0.1, 'n_estimators': 10}
28978.386707424626 {'learning_rate': 0.1, 'n_estimators': 50}
27174.70000800472 {'learning_rate': 0.1, 'n_estimators': 100}
